# Data México
## Importar bibliotecas

In [ ]:
import pandas as pd
#import matplotlib.pyplot as plt
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
import requests

## Leemos el archivo JSON del mapa de México

In [ ]:
url = 'https://raw.githubusercontent.com/angelnmara/geojson/master/mexicoHigh.json'
Mx_mapa_estados = requests.get(url).json()

In [ ]:
# Averiguamos la manera en la que están escritos los nombres de los estados
states = []
for i in range(32):
    states.append(Mx_mapa_estados["features"][i]["properties"]["name"])

In [ ]:
# Organizamos la lista de estados por orden alfabético
states.sort()
states

## Leemos los diferentes csv's

### Reporte de mobilidad por estados de México - Google

In [ ]:
mob_google = pd.read_csv("Databases/2020_MX_Region_Mobility_Report_Google.csv")

In [ ]:
mob_google.head(10)

In [ ]:
# Checamos la manera en la que están escritos los estados y corroboramos con nuestro mapa. El "nan" hace referencia a el país entero
mob_google["sub_region_1"].unique()

In [ ]:
# Reemplazamos los más obvios para luego comparar con la lista de estados del mapa
mob_google = mob_google.replace(["Mexico City", "State of Mexico"], ["Ciudad de México", "México"])
st_google = []
for i in range(1,33):
    st_google.append(mob_google["sub_region_1"].unique()[i])
st_google.sort()
for i in range(len(st_google)):
    print(st_google[i],states[i])
    # Mas adelante corroborara si los acentos influyen

In [ ]:
# Reemplazamos los nombres de los estados para que sean iguales a los del Mapa
for i in range(len(st_google)):
    mob_google = mob_google.replace(st_google[i], states[i])
mob_google["sub_region_1"].unique()

In [ ]:
# Cambiamos los nombres de las columnas a español y eliminamos columnas innecesarias
mob_google = mob_google.rename(columns= {'retail_and_recreation_percent_change_from_baseline': "Comercios y lugares recreativos",
                            'grocery_and_pharmacy_percent_change_from_baseline': "Farmacias y supermercados",
                            'parks_percent_change_from_baseline':"Parques",
                            'transit_stations_percent_change_from_baseline':'Estaciones de tránsito',
                            'workplaces_percent_change_from_baseline': "Lugares de trabajo",
                            'residential_percent_change_from_baseline':"Lugares residenciales"})
mob_google = mob_google.drop(columns=["sub_region_2", "metro_area", "iso_3166_2_code", "census_fips_code","country_region_code"])
mob_google

### Gráficas de movilidad durante la cuarentena para cada estado o país

Como primer paso hay que definir cuándo empezó la cuarentena en México. Según https://www.gob.mx/cms/uploads/attachment/file/541687/Jornada_Nacional_de_Sana_Distancia.pdf empézó el 23 de Marzo del 2020. Entonces hay que agregar una columna que nos permita discernir entre movilidad pre y durante la cuarentena.

In [ ]:
# Creamos una función que nos permita asignar el valor Durante Cuarentena y Antes de la Cuarentena
def cuarentena(fecha):
    # Documentación:
    # Evalúa la fecha de entrada y arroja si ésta ocurrió antes de la cuarentena o después  (23 de Marzo del 2020)
    
    # Argumentos:
    # fecha = e.g. 2020-03-23
    
    f_cuarentena = "2020-03-23"
    # Utilizamos un ciclo if para definir ambos casos
    if fecha < f_cuarentena:
        return "Antes de la Cuarentena"
    else:
        return "Durante la Cuarentena"
    
# Creamos la función que nos devuelva la gráfica
def movilidad_google(df,estado):
    # Documentación:
    # Toma el datafram de movilidad de google procesado y devuelve una gráfica dependiendo del estado que se eligió
    
    # Argumentos:
    # df = dataframe de movilidad de google procesado
    # estado = Nombre del estado (str)
    
    df["Cuarentena"] = df["date"].apply(cuarentena)
    if estado == "País":
        df_estado = df[df["sub_region_1"].isnull()]
    else:
        df_estado = df[df["sub_region_1"] == estado]
    
    import datetime
    df_estado["date"] = pd.to_datetime(df_estado["date"])
    df_estado.set_index("date",inplace=True)
    
    df_estado_2 = df_estado.resample("W").agg("mean")
    
    durante_la_cuarentena = df_estado["Cuarentena"].where(df_estado["Cuarentena"] == "Durante la Cuarentena").first_valid_index()
    antes_de_la_cuarentena = df_estado["Cuarentena"].where(df_estado["Cuarentena"] == "Antes de la Cuarentena").first_valid_index()
    mindate = df_estado.index.min()
    maxdate = df_estado.index.max()
    
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=[durante_la_cuarentena],
                             y=[39],
                             text=["<b>Empieza la Cuarentena</b>"],
                             mode="text"))
    
    l = df_estado_2.columns
    colors = px.colors.qualitative.G10
    
    for i in range(len(l)):
        fig.add_trace(go.Scatter(x=df_estado_2.index,
                                 y=df_estado_2[l[i]],
                                 mode="lines+markers",
                                 name=l[i],
                                 marker_color=colors[i])
                     )
        
        fig.update_layout(xaxis=dict(range=[mindate,maxdate]), template="plotly_white")
        
        fig.update_layout(title="Cambios en la Movilidad de los Habitantes del estado de {} durante la Cuarentena".format(estado),
                          width=1400,
                          height=600,
                          legend_title="Movilidad",
                          shapes=[dict(type="line",
                                       x0=mindate,
                                       y0=0,
                                       x1=maxdate,
                                       y1=0,
                                       line_width=2,
                                       line= dict(dash="dot")),
                                  dict(type="line",
                                       x0=durante_la_cuarentena,
                                       y0=-100,
                                       x1=durante_la_cuarentena,
                                       y1=37,
                                       line_width=2,
                                       line= dict(dash="dot"))
                                 ]
                         )
    return fig.show()

In [ ]:
vcm

In [ ]:
md

In [ ]:
def crime_lines(df, delito, estado):
    fig = go.Figure()
    for year in df["Año"].unique():
        fig.add_trace(go.Scatter(x=df["Mes"], y = df[(df["Tipo de delito"]==delito) & (df["Entidad"]==estado) & (df["Año"]==year)]["Incidencia"],mode="lines+markers",name=str(year)))
    fig.add_shape(type="line",x0="Marzo",x1="Marzo",y0=0,y1=df[(df["Tipo de delito"]==delito) & (df["Entidad"]==estado)]["Incidencia"].max(), layer="below", line=dict(width=5, dash="dot"))
    fig.add_shape(type="line",x0="Abril",x1="Abril",y0=0,y1=df[(df["Tipo de delito"]==delito) & (df["Entidad"]==estado)]["Incidencia"].max(), layer="below", line=dict(width=5, dash="dot"))
    fig.update_layout(title="{} en el estado de {} a través del tiempo".format(delito, estado),template="plotly_white")
    

    return fig.show()


In [ ]:
for estate in ["Baja California", "Guerrero"]:
    crime_lines(md, "Hostigamiento sexual", estate)

In [ ]:
fig = go.Figure()
for year in md["Año"].unique():
    fig.add_trace(go.Scatter(x=md["Mes"], y = md[(md["Tipo de delito"]=="Violencia familiar") & (md["Entidad"]=="Ciudad de México") & (md["Año"]==year)]["Incidencia"],mode="lines+markers",name=str(year)))
fig.add_shape(type="line",x0="Marzo",x1="Marzo",y0=0,y1=md[(md["Tipo de delito"]=="Violencia familiar") & (md["Entidad"]=="Ciudad de México") & (md["Año"]==year)]["Incidencia"].max(), layer="below", line=dict(width=5, dash="dot"))
fig.add_shape(type="line",x0="Abril",x1="Abril",y0=0,y1=md[(md["Tipo de delito"]=="Violencia familiar") & (md["Entidad"]=="Ciudad de México") & (md["Año"]==year)]["Incidencia"].max(), layer="below", line=dict(width=5, dash="dot"))
fig.update_layout(title="{} en {} a través del tiempo".format(delito, estado),template="plotly_white")
#fig.add_trace(go.Scatter(x=["2020-04-8"],y=[160], text="<b>Primer mes en</b>", mode="text", textfont_size=12))
#fig.add_trace(go.Scatter(x=["2020-04-8"],y=[150], text="<b>cuarentena</b>", mode="text", textfont_size=12))
#fig.add_trace(go.Scatter(x=["2020-04-8"],y=[140], text="23 de Marzo al", mode="text", textfont_size=10))
#fig.add_trace(go.Scatter(x=["2020-04-8"],y=[130], text="23 de Abril", mode="text", textfont_size=10))
fig.show()

In [ ]:
md[(md["Tipo de delito"]=="Violencia familiar") & (md["Entidad"]=="Yucatán") & (md["Año"]==2018)]

## Movilidad de México como país durante la Cuarentena

In [ ]:
# Mobilidad del país de México durante la Cuarentena
movilidad_google(mob_google, "País")

## Buscamos los valores más altos y bajos en el lapso de marzo 22 a abril 22 de tiempo en sus casas

In [ ]:
# Buscar los valores más altos entre el lapso de marzo 22 a abril 22 2020
mob_google_lapso = mob_google.loc[(mob_google["date"] >= "2020-03-22") & (mob_google["date"] <= "2020-04-22")] # Delimitamos la fecha a un mes 
mas_alto = mob_google_lapso["Lugares residenciales"].max() # Encontramos el valor más alto
mas_bajo = mob_google_lapso["Lugares residenciales"].min() # Encontramos el valor más bajo
estados_mas_t_casa = mob_google_lapso.loc[mob_google_lapso["Lugares residenciales"] >= mas_alto-2] # Buscamos los estados que cumplan con esta condición
estados_menos_t_casa = mob_google_lapso.loc[mob_google_lapso["Lugares residenciales"] <= mas_bajo+1] # Buscamos los estados que cumplan con esta condición
estados_menos_t_casa

#### Estados que permanecieron más tiempo en casa durante el primer mes de cuarentena

In [ ]:
for estado in estados_mas_t_casa["sub_region_1"].unique():
    movilidad_google(mob_google, estado)

#### Estados que permanecieron menos tiempo en casa durante el primer mes de cuarentena

In [ ]:
for estado in estados_menos_t_casa["sub_region_1"].unique():
    movilidad_google(mob_google, estado)

### Reporte de movilidad en México - Apple

In [ ]:
mob_apple = pd.read_csv("Databases/apple_mobility_trends-2020-11-25.csv")

In [ ]:
mob_apple.head(10)

In [ ]:
# Filtramos la tabla por los datos de México
mx_mob_apple = mob_apple.loc[mob_apple['region'] == "Mexico"]
mx_mob_apple

In [ ]:
# Eliminamos columnas innecesarias y con NA
mx_mob_apple = mx_mob_apple.drop(columns=["alternative_name", "sub-region", "country"])
mx_mob_apple = mx_mob_apple.dropna(axis=1) 
mx_mob_apple

In [ ]:
# Traducimos a español el tipo de transporte
mx_mob_apple = mx_mob_apple.replace(["driving","transit","walking"], ["Automóvil Propio", "Transporte Público", "Caminando"])
mx_mob_apple

In [ ]:
mx_mob_apple = mx_mob_apple.set_index(['geo_type', 'region', 'transportation_type']).stack().reset_index()
mx_mob_apple

In [ ]:
mx_mob_apple.columns = ["Geo_type", "Region", "Tipo de transporte", "Fecha", "Datos"]
mx_mob_apple

In [ ]:
mx_mob_apple[mx_mob_apple.isnull()]

In [ ]:
mx_mob_apple.fillna(0)

In [ ]:
fig = px.line(mx_mob_apple, x="Fecha", y= "Datos", color="Tipo de transporte", title="Reporte de movilidad en México")

fig.add_shape(type="line",x0="2020-03-23",x1="2020-03-23",y0=0,y1=170, layer="below", line=dict(width=1, dash="dot"))
fig.add_shape(type="line",x0="2020-04-23",x1="2020-04-23",y0=0,y1=170, layer="below", line=dict(width=1, dash="dot"))
fig.add_trace(go.Scatter(x=["2020-04-8"],y=[160], text="<b>Primer mes en</b>", mode="text", textfont_size=12))
fig.add_trace(go.Scatter(x=["2020-04-8"],y=[150], text="<b>cuarentena</b>", mode="text", textfont_size=12))
fig.add_trace(go.Scatter(x=["2020-04-8"],y=[140], text="23 de Marzo al", mode="text", textfont_size=10))
fig.add_trace(go.Scatter(x=["2020-04-8"],y=[130], text="23 de Abril", mode="text", textfont_size=10))

fig.show()

### Reporte de Delitos por Municipio del 2015 al 2020 - México

In [ ]:
md = pd.read_csv("Databases/Municipal-Delitos-2015-2020_oct2020.csv", encoding="latin_1")

In [ ]:
md.head(10)

In [ ]:
# Eliminamos columnas innecesarias
md = md.drop(columns=["Cve. Municipio", "Municipio", "Bien jurídico afectado", "Subtipo de delito", "Modalidad"])
md

In [ ]:
# Buscamos is hay NA
md.isnull().values.any()

In [ ]:
# Reemplazamos con NA con 0
md = md.fillna(0)

In [ ]:
# Verificamos si siguen habiendo NA
md.isnull().values.any()

In [ ]:
md

In [ ]:
# Convertimos valores flot a int para que se parezcan a los demás
md["Noviembre"] = md["Noviembre"].astype(int)
md["Diciembre"] = md["Diciembre"].astype(int)
md


In [ ]:
# Sumamos los tipos de delitos iguales (hay muchos debido a que eliminamos los subdelitos)
month_sum_dict = {"Enero":"sum","Febrero":"sum","Marzo":"sum","Abril":"sum","Mayo":"sum","Junio":"sum","Julio":"sum","Agosto":"sum","Septiembre":"sum","Octubre":"sum","Noviembre":"sum","Diciembre":"sum"}
md = md.groupby(["Año","Clave_Ent","Entidad","Tipo de delito"], as_index=False).agg(month_sum_dict)
md

In [ ]:
md = md.set_index(['Año', 'Clave_Ent', 'Entidad', "Tipo de delito"]).stack().reset_index()
md

In [ ]:
# Verificamos los estados en md sean iguales a los del mapa
md_st = []
for state in md["Entidad"].unique():
    md_st.append(state)
md_st.sort()
for i in range(len(states)):
    print(md_st[i],states[i])

In [ ]:
# Reemplazamos los nombres de los estados para que sean iguales a los del Mapa
for i in range(len(states)):
    md = md.replace(md_st[i], states[i])
md["Entidad"].unique()

In [ ]:
md

In [ ]:
md.columns = ["Año","Clave_Ent","Entidad","Tipo de delito","Mes","Incidencia"]

In [ ]:
md

In [ ]:
md[(md["Tipo de delito"]=="Homicidio") & (md["Año"]==2016) & (md["Mes"]=="Junio")]

In [ ]:
def pie_delitos(df, entidad, año, mes, crimenes):
    fig = px.pie(df[df["Tipo de delito"].isin(crimenes) & (df["Año"]==año) & (df["Entidad"]==entidad) & df["Mes"].isin(mes)],
                 values="Incidencia", names="Tipo de delito", hover_name="Mes", title="Crímenes en {} en los meses de {} y {} del año {}".format(entidad,mes[0],mes[1],año))
    return fig.show()

In [ ]:
for estado in estados_mas_t_casa["sub_region_1"].unique():
    pie_delitos(md, estado, 2020, ["Marzo","Abril"], ["Violencia familiar", "Abuso sexual", "Feminicidio", "Homicidio","Violación simple",
                                               "Acoso sexual", "Violación equiparada", "Aborto", "Hostigamiento sexual", "Incesto",
                                               "Violencia de género en todas sus modalidades distinta a la violencia familiar",
                                               "Otros delitos contra la familia", "Corrupción de menores",
                                               "Otros delitos que atentan contra la libertad y la seguridad sexual"])

In [ ]:
delitos_im = ["Violencia familiar", "Abuso sexual", "Feminicidio", "Homicidio","Violación simple",
                                               "Acoso sexual", "Violación equiparada", "Aborto", "Hostigamiento sexual", "Incesto",
                                               "Violencia de género en todas sus modalidades distinta a la violencia familiar",
                                               "Otros delitos contra la familia", "Corrupción de menores",
                                               "Otros delitos que atentan contra la libertad y la seguridad sexual"]

In [ ]:
def mapa_delitos(df, delito, mes):
    fig = px.choropleth(data_frame=df[(df["Tipo de delito"]==delito) & (md["Mes"]==mes)], 
                    geojson=Mx_mapa_estados, 
                    locations='Entidad',
                    featureidkey='properties.name',  # ruta al campo del archivo GeoJSON con el que se hará la relación (nombre de los estados)
                    color='Incidencia',
                    animation_frame="Año",
                    color_continuous_scale="burg",
                    height=800,
                    width=1200,
                    title="{} en {} a través de los años".format(delito, mes),
                    range_color=(0,df[(df["Tipo de delito"]==delito) & (md["Mes"]==mes)]["Incidencia"].max())
                    #scope="north america"
                   )
    fig.update_geos(showcountries=True, showcoastlines=False, showland=True, fitbounds="locations")

    return fig.show()

In [ ]:
for delito in delitos_im:
    mapa_delitos(md, delito, "Abril")

In [ ]:
# Filtrado de delitos
md_delitos = md.loc[md["Tipo de delito"].isin(["Violencia familiar", "Abuso sexual", "Feminicidio", "Homicidio","Violación simple",
                                               "Acoso sexual", "Violación equiparada", "Aborto", "Hostigamiento sexual", "Incesto",
                                               "Violencia de género en todas sus modalidades distinta a la violencia familiar",
                                               "Otros delitos contra la familia", "Corrupción de menores",
                                               "Otros delitos que atentan contra la libertad y la seguridad sexual"])]
md_delitos

In [ ]:
fig = px.pie(md_delitos[(md_delitos["Año"]==2019) & (md_delitos["Entidad"]=="Ciudad de México") & (md_delitos["level_4"]=="Enero")], values=0, names="Tipo de delito", hover_name="Año")
fig.show()

In [ ]:
# Renombramos las columnas
md_delitos.columns = ["Año", "Clave_Ent", "Entidad", "Tipo de delito", "Mes", "Incidencias"]
md_delitos[(md_delitos["Año"]==2015) & (md_delitos["Entidad"]=="Yucatán")]

In [ ]:
def line_delitos(df, entidad, delito):
    fig = px.line(df[(df["Tipo de delito"]==delito) & (df["Entidad"]==entidad)], x="Mes", y="Incidencia", animation_frame="Año",
                  title="{} a través de los años en el estado de {}".format(delito,entidad), range_y=[0,df[(df["Tipo de delito"]==delito) & (df["Entidad"]==entidad)]["Incidencia"].max()])
    return fig.show()

In [ ]:
def bar_delitos_1(df, entidad, delito):
    fig = px.bar(df[(df["Tipo de delito"]==delito) & (df["Entidad"]==entidad)], x="Año", y="Incidencia", color="Año", facet_col="Mes",facet_col_wrap=6, height=800, title="{} en el estado de {} a través del tiempo ".format(delito, entidad))
    return fig.show()

In [ ]:
bar_delitos_1(md, "Ciudad de México","Violencia familiar")

## Análisis de delitos a través del tiempo en los estados que permanecieron mas tiempo en casa

In [ ]:
for estado in estados_mas_t_casa["sub_region_1"].unique():
    for delito in md_delitos["Tipo de delito"].unique():
        bar_delitos_1(md, estado, delito)

## Análisis de delitos a través del tiempo en los estados que permanecieron menos tiempo en casa

In [ ]:
for estado in estados_menos_t_casa["sub_region_1"].unique():
    for delito in md_delitos["Tipo de delito"].unique():
        bar_delitos_1(md, estado, delito)

In [ ]:
# Definimos una función para hacer animaciones
def delitos_estado(df,estado, mes):
    # Documentación
    # Toma un datafram, estado y año como input y devuelve una animación de los delitos que ocurrieron en el estado durante ese año
    
    # Argumentos
    # df = dataframe preprocesado md_delitos
    # estado = estado de méxico (str)
    # año = un año (2015-2020) (int)
    fig = px.bar(df[df["Mes"].isin(mes) & (df["Entidad"]==estado)], x="Incidencias", y="Tipo de delito",
                 animation_frame="Año", title="Delitos en el estado de {} entre {} y {}  a través de los años".format(estado, mes[0],mes[1]),
                 range_x=[0,df[(df["Mes"]==mes[0]) & (df["Entidad"]==estado)]["Incidencias"].max()+df[(df["Mes"]==mes[1]) & (df["Entidad"]==estado)]["Incidencias"].max()], color="Mes",
                 height=1000)
                                                
    return fig.show()

In [ ]:
delitos_estado(md_delitos, "México", ["Marzo","Abril"])

In [ ]:
vcm = pd.read_csv("Databases/Info_delict_violencia_contra_la_mujer-2015_oct_2020.csv", encoding="latin_1")
vcm

In [ ]:
vcm = vcm.fillna(0)
vcm

In [ ]:
vcm = vcm.set_index(['Año', 'Mes', 'Feminicidios']).stack().reset_index()
vcm

In [ ]:
vcm.columns = ["Año", "Mes", "Feminicidios", "Tipo de delito", "Incidencias"]
vcm

In [ ]:
def bar_delitos(delito):
    fig = px.bar(vcm[vcm["Tipo de delito"]==delito], x="Año", y="Incidencias", color="Año", facet_col="Mes",facet_col_wrap=6, height=800, title="{} en México a través del tiempo ".format(delito))
    return fig.show()

In [ ]:
def crime_lines_2(df, delito):
    fig = go.Figure()
    for year in df["Año"].unique():
        fig.add_trace(go.Scatter(x=df["Mes"].unique(), y = df[(df["Tipo de delito"]==delito) & (df["Año"]==year)]["Incidencias"],mode="lines+markers",name=str(year)))
    fig.add_shape(type="line",x0="Marzo",x1="Marzo",y0=0,y1=df[(df["Tipo de delito"]==delito)]["Incidencias"].max(), layer="below", line=dict(width=5, dash="dot"))
    fig.add_shape(type="line",x0="Abril",x1="Abril",y0=0,y1=df[(df["Tipo de delito"]==delito)]["Incidencias"].max(), layer="below", line=dict(width=5, dash="dot"))
    fig.update_layout(title="{} en México a través del tiempo".format(delito),template="plotly_white")
    

    return fig.show()

In [ ]:
vcm["Mes"]

In [ ]:
fig = go.Figure()
for year in vcm["Año"].unique():
    fig.add_trace(go.Scatter(x=vcm["Mes"].unique(), y = vcm[(vcm["Tipo de delito"]=="Homicidio doloso") & (vcm["Año"]==year)]["Incidencias"]))
fig.show()

In [ ]:
for crime in vcm["Tipo de delito"].unique():
    crime_lines_2(vcm,crime)

In [ ]:
fig = go.Figure(data=[go.Bar(name=vcm[vcm["Tipo de delito"]=="Homicidio doloso"]["Año"], x=vcm[])])

In [ ]:
def violencia_mujer(df, mes):
    # Documentación
    # Toma un datafram, estado y año como input y devuelve una animación de los delitos que ocurrieron en el estado durante ese año
    
    # Argumentos
    # df = dataframe preprocesado md_delitos
    # estado = estado de méxico (str)
    # año = un año (2015-2020) (int)
    fig = px.bar(df[(df["Mes"]==mes)], x="Incidencias", y="Tipo de delito", animation_frame="Año",
                 title="Información sobre violencia contra las mujeres en México en el mes de {} a través de los años".format(mes),
                 range_x=[0,df[df["Mes"]==mes]["Incidencias"].max()])
                                                
    return fig.show()

In [ ]:
violencia_mujer(vcm, "Marzo")

In [ ]:
violencia_mujer(vcm, "Abril")

In [ ]:
def line_violencia(df, delito):
    fig = px.line(df[df["Tipo de delito"]==delito], x="Mes", y="Incidencias", color="Tipo de delito", animation_frame="Año",
                  title="{} en México a través de los años".format(delito),
                  range_y=[0,df[df["Tipo de delito"]==delito]["Incidencias"].max()])
    return fig.show()

In [ ]:
for delito in vcm["Tipo de delito"].unique():
    line_violencia(vcm, delito)